# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [31]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-llms-langchain

Defaulting to user installation because normal site-packages is not writeable


In [20]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings 
from sentence_transformers import SentenceTransformer

In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="llama-3.1-8b-instant",
)

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [3]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [33]:
documents = SimpleDirectoryReader('./data').load_data()
index = VectorStoreIndex.from_documents(documents)

In [41]:
# save index to disk
index.set_index_id("vector_index")
index.storage_context.persist("./storage")

In [4]:
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="storage")
# load index
index = load_index_from_storage(storage_context, index_id="vector_index")

In [55]:
query_engine = index.as_query_engine(
    similarity_top_k=5
)

In [56]:
from langchain_core.prompts import PromptTemplate

# Define the template that takes in the user question and the retrieved document
template = """
        You are an expert in HR topics. The user has a question, and you will answer it based on the relevant information provided from the HR manual.

        User question: {user_question}
        Relevant information: {answer}

        Provide a specific and accurate response based on the relevant information above.(NO PREAMBLE)
"""
# Create the PromptTemplate
prompt_template = PromptTemplate.from_template(
    template
)

In [57]:
question = "Can I take unpaid leave whenever I want?"
response = query_engine.query(question)
response.response

'No, according to the policy, if an employee exhausts their casual leave (10 days) and sick leave (13 days), any additional leave will be unpaid.'

In [62]:
response.source_nodes[4].text

"After this period, a decision will be made based on the device’s condition.\n205, Who is responsible for damage due to negligence when using a laptop outside the office?, If a laptop is damaged due to the user’s negligence outside the office, the user will be responsible, and a fine will be imposed.\n206, What is the policy for carrying laptops home?, Employees who take laptops home for office work must carry them at their own responsibility. If the laptop is lost or damaged, the user will bear the full cost of purchase or repair.\n207, Who decides whether an employee uses a desktop or a laptop?, The chairman will decide who is allowed to use a laptop. All other employees will use desktops, except for special cases.\n208, How are the official activities of Acme Technologies Limited managed?, All official activities of Acme Technologies Limited are managed through ERP software, with encrypted databases being recorded and automatically backed up to a server PC.\n209, What is the primary

In [54]:
chain = prompt_template | llm
res = chain.invoke(input={'user_question': question, 'answer': [response.source_nodes[0].text, response.source_nodes[1].text]})
print(res.content)

No, you cannot take unpaid leave whenever you want. According to the policy, if you exhaust your casual leave (10 days) and sick leave (13 days), any additional leave will be unpaid (Section 22).
